In [1]:
from model import *

In [2]:
from datetime import datetime

## Test the building and PV module

In [3]:
building1_csv = 'inputs/building1.csv'
building2_csv = 'inputs/building2.csv'
building3_csv = 'inputs/building3.csv'
pv1_csv = 'inputs/pv1.csv'
pv2_csv = 'inputs/pv2.csv'
pv3_csv = 'inputs/pv3.csv'

stepLenth = 15*60 # unit: s

building1 = Building(building1_csv, stepLenth)
building2 = Building(building2_csv, stepLenth)
building3 = Building(building3_csv, stepLenth)
pv1 = PV(pv1_csv, stepLenth)
pv2 = PV(pv2_csv, stepLenth)
pv3 = PV(pv3_csv, stepLenth)

In [4]:
timeIndex = building1.getLoadFullYear().index
timeStep = 50
print('At Time step {}, time is {}'.format(timeStep, timeIndex[timeStep]))
print('Building1 load is {}'.format(building1.getLoad(timeStep)))
print('PV1 generation is {}'.format(pv1.getPower(timeStep)))

At Time step 50, time is 2019-01-01 12:30:00
Building1 load is 0.2
PV1 generation is 5.75


## Test the vehicle module

In [26]:
fc1 = FuelCell(efficiency=90,  # 90kJ/g -- 25kWh/kg
               capacity=100)    # 100kW
fc2 = FuelCell(efficiency=90, capacity=100)  # 90kJ/g -- 25kWh/kg, 100kW

vehicle1_csv = 'inputs/vehicle1.csv'
vehicle2_csv = 'inputs/vehicle2.csv'
vehicle3_csv = 'inputs/vehicle3.csv'
schd_file = 'inputs/vehicle_atHomeSchd.csv'


vhc1 = Vehicle(vehicle1_csv, schd_file, 
               fuelCell=fc1, 
               stepLenth=stepLenth
               )

vhc2 = Vehicle(vehicle2_csv, schd_file, 
               fuelCell=fc2, 
               stepLenth=stepLenth
               )

vhc3 = Vehicle(vehicle3_csv, schd_file, 
               fuelCell=fc1, 
               stepLenth=stepLenth
               )

In [42]:
def test_h2FromStation():

    vhc1 = Vehicle(vehicle1_csv, schd_file, 
               fuelCell=fc1, 
               stepLenth=stepLenth
               )
    print(f'Tank Hydrogen after Initiation: {vhc1.tankVol}')
    print(f'Tank storage capacity: {vhc1.tankCapacity} g')
    print(f'Time step: {stepLenth} s')
    ## Charge the vehicle from station, case 1: small charge rate
    chargeRate = 1
    realChargeRate = vhc1.h2FromStation(chargeRate)
    print('------Case 1: small charge rate --------')
    print(f'Charge rate command - {chargeRate} g/s; actual - {realChargeRate} g/s')
    print(f'Charged hydrogen: {realChargeRate*stepLenth} g')
    print(f'Stored hydrogen in the tank: {vhc1.tankVol} g')
    
    ## Charge the vehicle from station, case 2: large charge rate
    print('------Case 2: large charge rate --------')
    vhc1 = Vehicle(vehicle1_csv, schd_file, 
               fuelCell=fc1, 
               stepLenth=stepLenth
               )
    chargeRate = 10
    realChargeRate = vhc1.h2FromStation(chargeRate)
    print(f'Charge rate command - {chargeRate} g/s; actual - {realChargeRate} g/s')
    print(f'Charged hydrogen: {realChargeRate*stepLenth} g')
    print(f'Stored hydrogen in the tank: {vhc1.tankVol} g')

In [43]:
test_h2FromStation()

Tank Hydrogen after Initiation: 0
Tank storage capacity: 5600.0 g
Time step: 900 s
------Case 1: small charge rate --------
Charge rate command - 1 g/s; actual - 1 g/s
Charged hydrogen: 900 g
Stored hydrogen in the tank: 900 g
------Case 2: large charge rate --------
Charge rate command - 10 g/s; actual - 6.222222222222222 g/s
Charged hydrogen: 5600.0 g
Stored hydrogen in the tank: 5600.0 g


In [29]:
def test_cruise(start_date):
    vhc2 = Vehicle(vehicle2_csv, schd_file, 
               fuelCell=fc1, 
               stepLenth=stepLenth
               )
    print(f'Tank storage capacity: {vhc1.tankCapacity} g')
    ## Charge the vehicle from station, case 1: small charge rate
    date_range = pd.date_range(start=start_date, periods=7, freq='1D')
    
    for date in date_range:
        weekend = date_range[0].dayofweek<5
        distance = vhc2._getDistance(weekend)
        if vhc2.tankVol < vhc2.fuelEff*distance:
            beforeCharge_tankStorage = vhc2.tankVol
            chargeRate = 100
            realChargeRate = vhc2.h2FromStation(chargeRate)
            afterCharge_tankStorage = vhc2.tankVol
            print(f'Charge the tank on: {date.strftime("%m-%d")}')
            print(f'  tank storage before charge {beforeCharge_tankStorage} g; after charge {afterCharge_tankStorage} g')
        beforeCruise_tankStorage = vhc2.tankVol
        vhc2.cruise(weekend)
        afterCruise_tankStorage = vhc2.tankVol
        print(f'Date {date.strftime("%m-%d")}, cruise distance: {distance} km;')
        print(f'  tank storage before cruise {beforeCruise_tankStorage} g; after cruise {afterCruise_tankStorage} g')


In [30]:
start_date = datetime(year=2021, month=1, day=1)
test_cruise(start_date)

Tank storage capacity: 5600.0 g
Charge the tank on: 01-01
  tank storage before charge 0 g; after charge 5600.0 g
Date 01-01, cruise distance: 37.075873728066874 km;
  tank storage before cruise 5600.0 g; after cruise 5263.499259735132 g
Date 01-02, cruise distance: 40.17748360656429 km;
  tank storage before cruise 5263.499259735132 g; after cruise 4875.626984981573 g
Date 01-03, cruise distance: 44.02392936485207 km;
  tank storage before cruise 4875.626984981573 g; after cruise 4483.675004709369 g
Date 01-04, cruise distance: 41.008325131476994 km;
  tank storage before cruise 4483.675004709369 g; after cruise 4155.160239484091 g
Date 01-05, cruise distance: 33.00600061056741 km;
  tank storage before cruise 4155.160239484091 g; after cruise 3803.7670172371 g
Date 01-06, cruise distance: 38.369913583499205 km;
  tank storage before cruise 3803.7670172371 g; after cruise 3491.72670507573 g
Date 01-07, cruise distance: 44.31344587823555 km;
  tank storage before cruise 3491.7267050757

In [40]:
def test_eleToGrid():
    vhc3 = Vehicle(vehicle3_csv, schd_file, 
               fuelCell=fc1, 
               stepLenth=stepLenth
               )
    print(f'Tank storage capacity: {vhc3.tankCapacity} g')
    print(f'Fuel cell efficiency {vhc3.fuelCell.getEff()} kJ/gH2; capacity: {vhc3.fuelCell.getCapacity()} kW')
    print(f'Time step: {stepLenth} s')
    
    chargeRate = 100
    realChargeRate = vhc3.h2FromStation(chargeRate)
    print(f'Stored hydrogen in the tank: {vhc3.tankVol} g')
    
    ## V2G, case 1: small charge rate
    prev_v2g = vhc3.tankVol
    dischargePower = 50  # kW
    realDischargeRate = vhc3.eleToGrid(dischargePower)
    after_v2g = vhc3.tankVol
    consumedH2_tank = prev_v2g-after_v2g
    consumedH2_power = realDischargeRate*stepLenth/vhc3.fuelCell.getEff()
    print('------Case 1: small charge rate --------')
    print(f'Charge rate: command - {dischargePower} kW; actual - {realDischargeRate} kW')
    print(f'Consumed H2: tank - {consumedH2_tank} g; power - {consumedH2_power} g')
    print(f'Stored hydrogen in the tank: before - {prev_v2g} g; after - {after_v2g} g')
    
    ## V2G, case 2: large charge rate
    prev_v2g = vhc3.tankVol
    dischargePower = 150  # kW
    realDischargeRate = vhc3.eleToGrid(dischargePower)
    after_v2g = vhc3.tankVol
    consumedH2_tank = prev_v2g-after_v2g
    consumedH2_power = realDischargeRate*stepLenth/vhc3.fuelCell.getEff()
    print('------Case 2: small discharge rate --------')
    print(f'Charge rate: command - {dischargePower} kW; actual - {realDischargeRate} kW')
    print(f'Consumed H2: tank - {consumedH2_tank} g; power - {consumedH2_power} g')
    print(f'Stored hydrogen in the tank: before - {prev_v2g} g; after - {after_v2g} g')
    
    ## V2G, case 3: low H2 storage
    vhc3.tankVol = 600
    prev_v2g = vhc3.tankVol
    dischargePower = 150  # kW
    realDischargeRate = vhc3.eleToGrid(dischargePower)
    after_v2g = vhc3.tankVol
    consumedH2_tank = prev_v2g-after_v2g
    consumedH2_power = realDischargeRate*stepLenth/vhc3.fuelCell.getEff()
    print('------Case 3: low storage --------')
    print(f'Charge rate: command - {dischargePower} kW; actual - {realDischargeRate} kW')
    print(f'Consumed H2: tank - {consumedH2_tank} g; power - {consumedH2_power} g')
    print(f'Stored hydrogen in the tank: before - {prev_v2g} g; after - {after_v2g} g')

In [41]:
test_eleToGrid()

Tank storage capacity: 5600.0 g
Fuel cell efficiency 90 kJ/gH2; capacity: 100 kW
Time step: 900 s
Stored hydrogen in the tank: 5600.0 g
------Case 1: small charge rate --------
Charge rate: command - 50 kW; actual - 50 kW
Consumed H2: tank - 500.0 g; power - 500.0 g
Stored hydrogen in the tank: before - 5600.0 g; after - 5100.0 g
------Case 2: small discharge rate --------
Charge rate: command - 150 kW; actual - 100 kW
Consumed H2: tank - 1000.0 g; power - 1000.0 g
Stored hydrogen in the tank: before - 5100.0 g; after - 4100.0 g
------Case 3: low storage --------
Charge rate: command - 150 kW; actual - 60.0 kW
Consumed H2: tank - 600.0 g; power - 600.0 g
Stored hydrogen in the tank: before - 600 g; after - 0.0 g


## Test H2 Station

In [ ]:
vhc1.tankVol

In [ ]:
vhc1.cruise(workingDay=True)

In [ ]:
vhc1.tankVol

In [ ]:
vhc1.eleToGrid(dischargePower=1000)

In [ ]:
vhc1.tankVol

In [ ]:
electrolyzer = Electrolyzer(efficiency=0.005, # 0.005g/kJ -- 56 kWh/kg
                            capacity=100      # g/s
                            )

In [ ]:
H2Station = H2Station(pipeChargingCapacity=5000,         # g/s
                      vehicleDischargingCapacity=1000,   # g/s
                      storageCapacity=1000*1000,         # g, data provided  
                      electrolyzer=electrolyzer, 
                      stepLenth=stepLenth
                     )

In [ ]:
vhc1.getParkSchd(False).plot()

In [ ]:
vhc1.h2FromStation(100)

In [ ]:
vhc1.tankVol

In [ ]:
vhc1.cruise(workingDay=True)

In [ ]:
vhc1.tankVol